In [1]:
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
from mriqc_learn.datasets import load_dataset
from mriqc_learn.models import preprocess as pp
from mriqc_learn.models.production import init_pipeline
from mriqc_learn.model_selection import split

## Load some data
We first load the ABIDE dataset, one of the default datasets distributed with MRIQC-learn

In [3]:
(train_x, train_y), (_, _) = load_dataset(split_strategy="none")
train_x["site"] = train_y.site

Let's pick the ratings from "rater_3" and binarize the three categories into only two.
We can also see that the dataset is unbalanced.

In [4]:
train_y = train_y[["rater_3"]].values.squeeze()
print(f"Discard={100 * (train_y == -1).sum() / len(train_y)}")
print(f"Doubtful={100 * (train_y == 0).sum() / len(train_y)}")
print(f"Accept={100 * (train_y == 1).sum() / len(train_y)}")
train_y[train_y == 0] = -1
train_y += 1
train_y[train_y > 0] = 1

Discard=14.168937329700272
Doubtful=1.5440508628519527
Accept=84.28701180744777


Let's print out a pretty view of the data table:

In [5]:
train_x

,cjv,cnr,efc,fber,fwhm_avg,fwhm_x,fwhm_y,fwhm_z,icvs_csf,icvs_gm,...,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max,site
0,0.383747,3.259968,0.609668,181.619858,3.944888,3.959924,4.039157,3.835584,0.199774,0.449138,...,1000.013428,189965.0,908.938904,1079.413428,51.778980,0.225944,0.525072,0.540801,0.540213,PITT
1,0.574080,2.279440,0.606361,172.500031,3.992397,3.877495,4.173095,3.926602,0.203301,0.429628,...,1000.033569,187992.0,901.788293,1120.833569,67.136932,0.223374,0.521399,0.560238,0.571425,PITT
2,0.314944,3.998569,0.577123,273.688171,4.016382,4.066009,4.092888,3.890248,0.201591,0.446495,...,1000.015198,188213.0,913.847803,1067.003662,46.623932,0.233414,0.531020,0.556496,0.612655,PITT
3,0.418505,3.050534,0.571343,237.531143,3.601741,3.629409,3.627568,3.548246,0.190612,0.468255,...,1000.005981,146722.0,872.409717,1083.139264,63.131420,0.227282,0.528115,0.526254,0.600312,PITT
4,0.286560,4.214082,0.550083,427.042389,3.808350,3.839143,3.841085,3.744823,0.162421,0.505201,...,1000.004150,162584.0,900.433481,1069.912750,50.874363,0.195150,0.543591,0.531606,0.603308,PITT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,0.428731,3.030323,0.789654,2519.999512,3.176760,3.166740,3.359990,3.003550,0.169851,0.424819,...,1000.034668,241117.0,902.529590,1088.244409,56.683868,0.162535,0.476992,0.536843,0.537140,SBL
1097,0.610845,2.155928,0.800116,1769.720093,3.209497,3.164760,3.381280,3.082450,0.170732,0.405536,...,1000.039429,251136.0,903.951080,1093.323273,57.789230,0.193376,0.465232,0.545695,0.564010,SBL
1098,0.461773,2.794299,0.789859,2248.858398,3.149920,3.112220,3.326700,3.010840,0.165501,0.441190,...,1000.036438,209298.0,891.934216,1093.973322,61.108639,0.198508,0.497137,0.523571,0.564865,SBL
1099,0.457718,2.862913,0.706924,114.865364,3.486750,3.421200,3.881950,3.157100,0.209701,0.381839,...,999.990356,234957.0,904.907922,1101.429980,60.045422,0.235618,0.477310,0.563352,0.534626,MAX_MUN


## Cross-validation of the default classifier
Let's cross-validate the performance of our classifier using a Leave-one-site-out strategy.

In [6]:
# Define a splitting strategy
outer_cv = split.LeavePSitesOut(1, robust=True)
# Initiate the default model
model = init_pipeline()

We can now feed the model into the cross-validation loop:

In [7]:
score = cross_val_score(
    model,
    X=train_x,
    y=train_y,
    cv=outer_cv,
    scoring="roc_auc",
    n_jobs=16,
)

After one or two minutes, the scores have been caculated for each of the 14 folds our splitter created.
The average performance is AUC=0.885.

In [8]:
print(score)
score.mean()

[0.77157895 0.89615385 0.91848265 0.98333333 0.43243243 1.
 0.96938776 1.         0.89922481 1.         0.90402477 0.91823899
 0.84433962 0.76170213]


0.8784928056929616

We now train the model on all available training data:

In [9]:
model = model.fit(
    X=train_x,
    y=train_y,
)    

We can easily see the effects of overfitting by evaluating the classifier on the same folds we used for cross-validation.

In [10]:
from sklearn.metrics import roc_auc_score as auc

trained_scores = {}
for train_index, test_index in split.LeavePSitesOut(1, robust=True).split(train_x):
    test_X = train_x.iloc[test_index]
    test_y = train_y[test_index]
    if np.unique(test_y).size > 1:
        site = tuple(set(test_X.site.values.tolist()))
        trained_scores[site] = auc(test_y, model.predict(test_X))
np.mean(list(trained_scores.values()))

0.7893888953757795

In [11]:
from sklearn.metrics import classification_report

print(classification_report(train_y, model.predict(train_x)))

              precision    recall  f1-score   support

           0       0.92      0.40      0.56       173
           1       0.90      0.99      0.94       928

    accuracy                           0.90      1101
   macro avg       0.91      0.70      0.75      1101
weighted avg       0.90      0.90      0.88      1101



## Evaluating on held-out dataset
We first load the held-out dataset in, and evaluate:

In [12]:
(test_x, test_y), (_, _) = load_dataset("ds030", split_strategy="none")
test_x["site"] = test_y.site
test_x

,cjv,cnr,efc,fber,fwhm_avg,fwhm_x,fwhm_y,fwhm_z,icvs_csf,icvs_gm,...,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max,site
0,0.550186,2.459577,0.507058,1065.732178,3.480117,3.507830,3.70268,3.22984,0.243019,0.395910,...,1000.057861,183285.0,883.837659,1149.885156,81.690285,0.215809,0.476166,0.560368,0.677318,BMC
1,0.456006,2.921997,0.561604,799.381470,3.321400,3.347790,3.50000,3.11641,0.175314,0.463107,...,1000.007141,193080.0,892.560648,1105.589551,64.899048,0.233023,0.535007,0.564211,0.649004,BMC
2,0.445959,2.908593,0.549342,1123.041870,3.137148,3.138153,3.35693,2.91636,0.174694,0.411513,...,999.994690,205413.0,903.285217,1099.551831,59.837898,0.230147,0.492765,0.577273,0.611447,BMC
3,0.767179,1.766171,0.568210,630.778992,3.339373,3.364539,3.45910,3.19448,0.209167,0.389043,...,1000.085632,255944.0,879.354327,1157.930383,85.289146,0.228836,0.474763,0.554115,0.630658,BMC
4,0.539929,2.473583,0.541802,937.298462,3.108547,3.133720,3.26695,2.92497,0.198797,0.442200,...,1000.033508,170103.0,894.828833,1112.822083,66.272682,0.201488,0.497095,0.538423,0.661969,BMC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,0.433127,3.111099,0.562418,886.391785,3.352552,3.442397,3.51022,3.10504,0.198912,0.418726,...,999.999268,236049.0,896.524268,1102.766406,62.811314,0.208632,0.502756,0.565940,0.651403,CCN
261,0.455297,2.946393,0.494483,985.328186,3.015977,3.059440,3.12296,2.86553,0.207224,0.421287,...,1000.008179,152038.0,905.155991,1092.722668,57.217701,0.172909,0.490781,0.573420,0.667625,CCN
262,0.428027,3.056015,0.544575,899.129028,3.158103,3.193888,3.29329,2.98713,0.188024,0.418417,...,999.986328,200700.0,904.591296,1087.129657,55.752438,0.222306,0.495817,0.566268,0.669586,CCN
263,0.456340,2.968104,0.531945,963.673828,3.203233,3.230830,3.41218,2.96669,0.184849,0.435990,...,1000.027588,162744.0,898.410342,1099.041461,61.291889,0.214909,0.495231,0.545337,0.612139,CCN


In [13]:
has_ghost = test_y.has_ghost.values.astype(bool)
test_y = test_y[["rater_1"]].values.squeeze()
print(f"Discard={100 * (test_y == -1).sum() / len(test_y)}")
print(f"Doubtful={100 * (test_y == 0).sum() / len(test_y)}")
print(f"Accept={100 * (test_y == 1).sum() / len(test_y)}")
test_y[test_y == 0] = -1
test_y += 1
test_y[test_y > 0] = 1

Discard=28.30188679245283
Doubtful=54.716981132075475
Accept=16.9811320754717


In [14]:
auc(test_y, model.predict(test_x))

0.5409090909090909

In [15]:
auc(test_y[~has_ghost], model.predict(test_x[~has_ghost]))

0.5348837209302326

In [16]:
print(classification_report(test_y, model.predict(test_x)))

              precision    recall  f1-score   support

           0       1.00      0.08      0.15       220
           1       0.18      1.00      0.31        45

    accuracy                           0.24       265
   macro avg       0.59      0.54      0.23       265
weighted avg       0.86      0.24      0.18       265



In [17]:
print(classification_report(test_y[~has_ghost], model.predict(test_x[~has_ghost])))

              precision    recall  f1-score   support

           0       1.00      0.07      0.13       172
           1       0.22      1.00      0.36        45

    accuracy                           0.26       217
   macro avg       0.61      0.53      0.25       217
weighted avg       0.84      0.26      0.18       217



## Nested cross-validation

In [ ]:
p_grid = [{
    "scale__unit_variance": [True, False],
    "scale__with_centering": [True, False],
    "site_pred__disable": [False, True],
    "winnow__disable": [False, True],
    "svc__kernel": ["rbf"],
    "svc__C": [10],
    "svc__gamma": [0.1],
}]

In [ ]:
# Nested CV with parameter optimization
inner_cv = split.LeavePSitesOut(1, robust=True)
inner_cv.get_n_splits(X=train_x, y=train_y)

clf = GridSearchCV(
    estimator=pipe,
    param_grid=p_grid,
    cv=inner_cv,
    n_jobs=30,
    scoring="roc_auc",
)
# clf.fit(train_x, y=train_y)

In [ ]:
nested_score = cross_val_score(
    clf,
    X=train_x,
    y=train_y,
    cv=outer_cv,
    scoring="roc_auc",
    verbose=10,
    n_jobs=16,
)
nested_score.mean()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_